# NoSQL DBs in Generative AI

**MongoDB:** Used for storing structured/unstructured metadata, user prompts, model outputs, and other app-level data.

**Elasticsearch:** Used for semantic search or retrieval-augmented generation (RAG), where relevant context is retrieved and passed to the LLM to enhance response quality.

## AI Knowledge Assistant

🗣️ Accepts a user query

🔍 Uses an Elasticsearch retriever tool to fetch related docs

🧾 Optionally checks user history or context from MongoDB

🤖 Uses LangChain Agent to decide how to answer

💬 Generates a response using LLM

🧠 Stores chat in MongoDB


**Architecture**

User Query

   ↓

LangChain Agent

   ├── Tool 1: ElasticsearchRetrieverTool

   ├── Tool 2: MongoDBChatHistoryTool

   ↓

LLM generates answer using both tools

   ↓

Answer is saved to MongoDB


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import traceback

from langchain.agents import initialize_agent
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from pymongo import MongoClient

In [4]:
INDEX_NAME = "product_docs_vector"

documents = [
    "To reset your password, go to the settings page and click on 'Forgot Password'.",
    "Our refund policy allows returns within 30 days of purchase, with the original receipt.",
    "All electronics come with a one-year manufacturer warranty unless specified otherwise.",
    "You can track your order using the tracking number sent to your email after shipping.",
    "Contact our support team 24/7 via live chat or by calling our toll-free number.",
    "Premium members receive free shipping and priority customer support.",
    "Your data is encrypted and stored securely as per industry standards.",
    "Multiple failed login attempts will temporarily lock your account for 15 minutes.",
    "You can change your communication preferences from your profile settings.",
    "Install our mobile app for a better experience and faster checkout."
]

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name="thenlper/gte-small"
)

In [6]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_texts(texts=documents, embedding=embedding)

In [7]:
mongo = MongoClient("mongodb://localhost:27017/")
db = mongo["support_bot"]
chat_collection = db["chats"]

In [8]:
llm = ChatOpenAI(model="gpt-4", verbose=True)

In [51]:
# ==== Tool 1: Hybrid FAISS Retriever ====
class HybridFAISSRetrieverTool(Tool):
    name: str = "HybridFAISSRetrieverTool"
    description: str = "Use this tool to retrieve relevant documents from the FAISS vector store based on the user query. It uses a hybrid approach combining vector search and semantic search."
    
    # tool class attributes should be defined here
    _vectorstore: FAISS
    embedding_model: HuggingFaceEmbeddings

    def __init__(self, _vectorstore: FAISS, embedding_model: HuggingFaceEmbeddings, name: str = None, description: str = None):
        self._vectorstore = _vectorstore
        self.embedding_model = embedding_model
        self.name = name or self.name
        self.description = description or self.description
        super().__init__(name=name or self.name, func=self._perform_action, description=description or self.description)

    def _preform_action(self, query: str) -> str | None:
        try:
            # Generate embedding for the user query
            query_vector = self.embedding_model.embed_query(query)

            # Perform similarity search in FAISS
            results = self._vectorstore.similarity_search_by_vector(query_vector, k=5)

            if not results:
                return "No relevant content found."

            _response = []
            for result in results:
                content = result.page_content
                score = result.score  # similarity score
                _response.append(f"Score: {score:.2f}\nContent:\n{content}")

            return "\n\n---\n\n".join(_response)

        except Exception as e:
            return f"Error during hybrid FAISS search: {str(e)}\n{traceback.format_exc()}"


In [52]:
# ==== Tool 2: MongoDB Chat History ====
class MongoChatHistoryTool(Tool):
    name: str = "MongoChatHistoryTool"
    description: str = "Use this tool to retrieve the last 3 chat history entries from MongoDB."

    def __init__(self, name: str = None, description: str = None):
        super().__init__(name=name or self.name, description=description or self.description, func=self._preform_action)

    @staticmethod
    def _preform_action(user_id: str):
        history = chat_collection.find({"user_id": user_id}).sort("timestamp", -1).limit(3)
        return "\n".join([f"User: {h['prompt']}\nBot: {h['response']}" for h in history])

In [ ]:
faiss_tool = HybridFAISSRetrieverTool(_vectorstore=vectorstore, embedding_model=embedding)
mongo_tool = MongoChatHistoryTool()

tools = [
    faiss_tool,
    mongo_tool
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True,
    return_intermediate_steps=True
)


In [ ]:
def generate_response(user_id, prompt):
    system_prompt = f"The user ID is {user_id}. The user asked: '{prompt}'"
    final_response = agent.run(system_prompt)

    # Save to MongoDB
    chat_collection.insert_one({
        "user_id": user_id,
        "prompt": prompt,
        "response": final_response
    })

    return final_response

In [ ]:
response = generate_response(user_id="123", prompt="How do I reset my password?")
print(response)